# Technique: 10 Aggregation

### What is this?
Aggregation means replacing many small, detailed records with a few summary numbers. For example, instead of every bus stop event, we store one total number for the whole hour.

### Why use it?
1. **Save Space**: It costs a lot of money to store millions of rows.
2. **Faster Analysis**: It is much faster to check a monthly total than a million daily rows.
3. **See Trends**: It helps us see the "Big Picture" (like the morning rush hour).

### Methods:
* **Time Aggregation**: Grouping data by minute, hour, day, or week.
* **Aggregate Functions**: Using math like `SUM()`, `AVERAGE()`, or `COUNT()`.

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from data_generator import generate_dtt_dataset, GLOBAL_SEED

# 1. Get the data
df = generate_dtt_dataset(n_samples=1000)

# 2. Create a fake "Timestamp" column for 1000 transactions
# We use 'min' instead of 'T' for minutes
df['Timestamp'] = pd.date_range(start='2025-06-13', periods=1000, freq='min')

# 3. Look at the detailed data
print("Detailed Data (Per Minute):")
print(df[['Timestamp', 'Transaction_Amount']].head())

Detailed Data (Per Minute):
            Timestamp  Transaction_Amount
0 2025-06-13 00:00:00            3.257499
1 2025-06-13 00:01:00           68.050678
2 2025-06-13 00:02:00            8.111841
3 2025-06-13 00:03:00           27.766274
4 2025-06-13 00:04:00           13.867245


## Grouping the Data
We will "roll up" the per-minute data into **Hourly** summaries. We will use `SUM` to find the total money and `MEAN` to find the average.

In [9]:
# 1. Set the Timestamp as the index to use time tools
# Note: Only run this once. If 'Timestamp' is already the index, skip this line.
if df.index.name != 'Timestamp':
    df.set_index('Timestamp', inplace=True)

# 2. Resample to 'h' (Hourly) and calculate summaries
# We use lowercase 'h' instead of 'H'
df_hourly = df['Transaction_Amount'].resample('h').agg(['sum', 'mean', 'count'])

# 3. Check the results
print("Aggregated Data (Per Hour):")
print(df_hourly.head())

# 4. Compare the size
print(f"\nOriginal rows: {len(df)}")
print(f"Reduced (Aggregated) rows: {len(df_hourly)}")

Aggregated Data (Per Hour):
                             sum       mean  count
Timestamp                                         
2025-06-13 00:00:00  1791.047133  29.850786     60
2025-06-13 01:00:00  1757.387171  29.289786     60
2025-06-13 02:00:00  2134.974297  35.582905     60
2025-06-13 03:00:00  2315.036339  38.583939     60
2025-06-13 04:00:00  2634.622695  43.910378     60

Original rows: 1000
Reduced (Aggregated) rows: 17


## Grouping by Category
We can also aggregate by other groups, like **Education_Level**, to see the average salary for each group.

In [10]:
# Group by Education and find the Average Salary
edu_summary = df.groupby('Education_Level')['Annual_Salary'].mean()

print("Average Salary by Education Level:")
print(edu_summary)

Average Salary by Education Level:
Education_Level
Bachelor       51945.814646
High School    50497.493868
Master         52870.334908
PhD            51256.407442
Name: Annual_Salary, dtype: float64


### Summary from Lecture Slides:
* **Storage Space**: We save a lot of storage by not keeping every tiny detail forever.
* **Efficiency**: Moving from "per-minute" to "hourly" makes the dataset smaller and more valuable for strategy.
* **Trade-off**: We lose the very small details, but we gain a lot of speed and clear trends.

This process moves us from "Raw Data" to "Strategic Information"!